This is a cover letter builder using agentic ai


In [52]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [53]:
load_dotenv(override=True)


True

In [54]:
openai = OpenAI()
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [55]:
creator_model = "gpt-4o-mini"
evaluator_model = "gemini-2.0-flash"

In [56]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [57]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [58]:
name = "Sviatoslav Rutkovskyi"


In [59]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use to answer questions. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
"""


In [60]:
evaluator_system_prompt = f"""
You are an evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. 
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. 
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [61]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [62]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your feedback.
    """

In [63]:
def update_system_prompt(cover_letter, feedback):
        return system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """

In [64]:
def evaluate(job_post, cover_letter) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)}]
    response = gemini.beta.chat.completions.parse(model=evaluator_model, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [65]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [66]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [71]:
def requestLetter(job_posting, history):
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 5:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [72]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Failed evaluation - retrying


## Cover Letter:
[Today’s Date]

Hiring Manager  
GForce Life Sciences  
Redmond, WA  

Dear Hiring Manager,

I am writing to express my interest in the Entry-Level Software Engineer position at GForce Life Sciences, as advertised. With a Bachelor of Science in Computer Science from the University of Washington Tacoma, where I achieved a 3.9 GPA, I am eager to contribute my knowledge and skills to your innovative team while making a positive impact in the field of life sciences.

Throughout my academic journey, I have developed a robust foundation in software development, enriched by practical projects that demonstrate my ability to design, develop, and implement software components effectively. For instance, I led a team to create a full-stack web application for managing a book catalog, where I took responsibility for designing a RESTful API backend and navigating complex problems through teamwork and communication. This project honed my skills in Node.js, Express, and PostgreSQL, while also providing me with hands-on experience in maintaining project standards and ensuring code quality through testing.

In addition to my technical acumen, my experience as a Sergeant in the US Army Reserves has equipped me with remarkable leadership and teamwork abilities, enabling me to contribute effectively in high-pressure environments. I have learned the importance of integrity, discipline, and the commitment to achieving project goals—all of which I aim to bring to the Software Engineer role at GForce Life Sciences.

I am particularly drawn to your company’s commitment to leveraging technology to improve healthcare solutions. I am passionate about utilizing my programming skills in areas like AI and machine learning, which I believe can transform the way we approach life sciences. I am excited about the opportunity to work in a dynamic and collaborative setting where I can apply my expertise and engage in meaningful projects.

Thank you for considering my application. I am looking forward to the opportunity to discuss how my background, skills, and ambitions align with the goals of GForce Life Sciences. I am enthusiastic about the possibility of contributing to your team and am eager to bring my passion for software development to this exciting position.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
The cover letter is well-written and tailored to the job description. It appropriately highlights your academic achievements, relevant projects, and military experience, and includes some of the keywords mentioned in the job description. The letter also expresses enthusiasm for the company's mission and the opportunity to contribute to their team. However, the language used in the cover letter seems a bit generic and could be improved to sound more authentic and less like it was generated by AI. Also, you should remove the line with Today's Date, and replace it with the actual date.

Passed evaluation - returning reply
